### Part 1

##### State the null hypothesis

μ1 = μ2 = μ3

##### State the alternate hypothesis

Not μ1 = μ2 = μ3

##### What is the significance level

α = 0.05

##### What are the degrees of freedom of model, error terms, and total DoF

df1 (degrees of freedom) = 2 \
df2 (error terms) = 12 \
df_total (total degrees of freedom) = 14

### Part 2

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_excel('anova_lab_data.xlsx', sheet_name='data_collected')
data

,Power,Etching Rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [3]:
data = data.rename(columns=lambda x: x.lower().replace(' ', '_'))
data

,power_,etching_rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [4]:
data = data.rename(columns={'power_': 'power'})
data

,power,etching_rate
0,160 W,5.43
1,180 W,6.24
2,200 W,8.79
3,160 W,5.71
4,180 W,6.71
5,200 W,9.20
6,160 W,6.22
7,180 W,5.98
8,200 W,7.90
9,160 W,6.01


In [7]:
data['power'] = data['power'].str.replace(' W', '')
data

,power,etching_rate
0,160,5.43
1,180,6.24
2,200,8.79
3,160,5.71
4,180,6.71
5,200,9.20
6,160,6.22
7,180,5.98
8,200,7.90
9,160,6.01


In [8]:
data.dtypes

power            object
etching_rate    float64
dtype: object

In [9]:
data['power'] = data['power'].astype(int)

In [10]:
data.describe()

,power,etching_rate
count,15.000000,15.000000
mean,180.000000,6.782667
std,16.903085,1.228643
min,160.000000,5.430000
25%,160.000000,5.845000
50%,180.000000,6.240000
75%,200.000000,7.725000
max,200.000000,9.200000


In [12]:
data.groupby('power').agg(np.mean)

,etching_rate
power,
160,5.792
180,6.238
200,8.318


We can see that power = 200 seems to be the more different mean out of all. But first, we'll use ANOVA to determine if in fact these observations are different in a statistical significant way. 

In [13]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('etching_rate ~ C(power)',data=data).fit()
sm.stats.anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(power),2.0,18.176653,9.088327,36.878955,0.000008
Residual,12.0,2.957240,0.246437,NaN,NaN


Our ANOVA test P value is lower than our alpha of 0.05, meaning that we can reject the null hypothesis. As such, we know that there is a statistically significant difference between the mean of the etching rate observed at different temperatures. 

Now we will try to find the source of the difference between these measures, focusing on power = 200 and we'll conduct T tests between this category and all the others to see if we can prove that they are statistically equal or not. In this case, our null hypothesis is that they are equal and our alternative hypothesis is that they are not. 

In [16]:
from scipy.stats import ttest_ind

power_a = data[data['power'] == 200]['etching_rate']

for power in data['power'].unique():
    power_b = data[data['power'] == power]['etching_rate']
    print(power, ttest_ind(power_a, power_b))

160 Ttest_indResult(statistic=7.611403634613074, pvalue=6.237977344615716e-05)
180 Ttest_indResult(statistic=5.827496614588661, pvalue=0.0003926796476049085)
200 Ttest_indResult(statistic=0.0, pvalue=1.0)


We can see that our P values are below our alpha of 0.05, so we reject the null hypothesis meaning that power = 200 is statistically different from the rest and proving it had an effect on the etching rate.